In [1]:
import pickle
import random
import time
from collections import defaultdict, Counter
from tqdm import tqdm

### --- Step 1: Load Passwords --- ###
def load_passwords(file_path, max_lines=10000):
    with open(file_path, 'r', encoding='latin-1', errors='ignore') as f:
        return [line.strip() for i, line in enumerate(f) if line.strip() and i < max_lines]

### --- Step 2: Train Higher-order Markov Model (3-gram) --- ###
def train_markov_model(passwords, state_size=3):
    model = defaultdict(list)
    for password in passwords:
        for i in range(len(password) - state_size):
            state = password[i:i + state_size]
            next_char = password[i + state_size]
            model[state].append(next_char)
    return dict(model)

### --- Step 3: Weighted Character Sampling --- ###
def weighted_choice(char_list):
    counter = Counter(char_list)
    chars, weights = zip(*counter.items())
    return random.choices(chars, weights=weights)[0]

### --- Step 4: Weighted Markov Password Generator --- ###
def generate_weighted_password(model, seed="pas", length=10):
    password = seed
    while len(password) < length:
        state = password[-len(seed):]
        next_chars = model.get(state)
        if not next_chars:
            break
        next_char = weighted_choice(next_chars)
        password += next_char
    return password

### --- Step 5: Cracking Attempt Function --- ###
def crack_password_with_model(target, model, length, max_attempts=1000000, seeds=None):
    attempts = 0
    tried = set()
    start = time.time()

    if not seeds:
        seeds = [target[:3], target[:2], "pas", "lov", "123"]

    for seed in seeds:
        while attempts < max_attempts:
            gen = generate_weighted_password(model, seed=seed, length=length)
            attempts += 1
            if gen == target:
                return True, attempts, time.time() - start
            tried.add(gen)
            if attempts % 100000 == 0:
                print(f"Attempts: {attempts}...")

    return False, attempts, time.time() - start

### --- Step 6: Evaluator Function --- ###
def evaluate_model(passwords, model, sample_size=100, max_attempts=1000000):
    cracked = 0
    total_attempts = 0
    total_time = 0

    sample = random.sample(passwords, min(sample_size, len(passwords)))

    for pwd in tqdm(sample, desc="🔍 Evaluating Model"):
        length = len(pwd)
        success, attempts, time_taken = crack_password_with_model(
            target=pwd,
            model=model,
            length=length,
            max_attempts=max_attempts,
            seeds=[pwd[:3], pwd[:2], "pas", "lov", "123"]
        )
        if success:
            cracked += 1
            total_attempts += attempts
            total_time += time_taken

    accuracy = cracked / sample_size
    avg_attempts = total_attempts / cracked if cracked > 0 else float('inf')
    avg_time = total_time / cracked if cracked > 0 else float('inf')

    print("\n📊 Evaluation Results")
    print(f"Top-{max_attempts} Accuracy: {accuracy * 100:.2f}%")
    print(f"Average Attempts (cracked only): {avg_attempts:.2f}")
    print(f"Average Time per Crack: {avg_time:.2f} sec")

### --- Step 7: Train, Save, Load, and Evaluate --- ###

# Load training data
passwords = load_passwords("rockyou.txt", max_lines= 50000)  # Feel free to increase

# Train 3-gram model
print("🛠️ Training 3-gram Markov model...")
model = train_markov_model(passwords, state_size=3)

# Save model
with open("markov_model_3gram.pkl", "wb") as f:
    pickle.dump(model, f)
print("✅ Model saved as markov_model_3gram.pkl")

# Evaluate model
evaluate_model(passwords, model, sample_size=100, max_attempts=1000000)


🛠️ Training 3-gram Markov model...
✅ Model saved as markov_model_3gram.pkl


🔍 Evaluating Model:  13%|█▎        | 13/100 [00:06<00:34,  2.51it/s]

Attempts: 100000...


🔍 Evaluating Model:  13%|█▎        | 13/100 [00:20<00:34,  2.51it/s]

Attempts: 200000...
Attempts: 300000...
Attempts: 400000...
Attempts: 500000...


...

Attempts: 100000...
Attempts: 200000...


🔍 Evaluating Model:  88%|████████▊ | 88/100 [03:34<00:06,  1.87it/s]

Attempts: 100000...


🔍 Evaluating Model: 100%|██████████| 100/100 [03:43<00:00,  2.23s/it]


📊 Evaluation Results
Top-1000000 Accuracy: 100.00%
Average Attempts (cracked only): 31880.08
Average Time per Crack: 2.23 sec


In [8]:
import pickle
import random
import time
from collections import Counter

### --- Load the 3-gram Model --- ###
with open("markov_model_3gram.pkl", "rb") as f:
    model = pickle.load(f)

### --- Weighted Sampling --- ###
def weighted_choice(char_list):
    counter = Counter(char_list)
    chars, weights = zip(*counter.items())
    return random.choices(chars, weights=weights)[0]

### --- Password Generator --- ###
def generate_password(model, seed="pas", length=10):
    password = seed
    while len(password) < length:
        state = password[-len(seed):]
        next_chars = model.get(state)
        if not next_chars:
            break
        password += weighted_choice(next_chars)
    return password

### --- Cracking Function --- ###
def crack_password(target, model, max_attempts=100000000):
    length = len(target)
    seeds = [target[:3], target[:2], "pas", "lov", "123", "qwe"]
    tried = set()
    attempts = 0
    start = time.time()

    for seed in seeds:
        while attempts < max_attempts:
            pwd = generate_password(model, seed=seed, length=length)
            attempts += 1
            if pwd == target:
                end = time.time()
                print(f"\n✅ Password cracked!")
                print(f"🔐 Password: {pwd}")
                print(f"📊 Attempts: {attempts}")
                print(f"⏱️ Time taken: {end - start:.2f} seconds")
                return pwd
            tried.add(pwd)
            if attempts % 100000 == 0:
                print(f"Still trying... {attempts} attempts")

    print("\n❌ Failed to crack the password within attempt limit.")
    return None

### --- User Input + Crack --- ###
target_password = input("🔑 Enter a password to crack: ").strip()
crack_password(target_password, model)


🔑 Enter a password to crack:  lovely123



✅ Password cracked!
🔐 Password: lovely123
📊 Attempts: 17
⏱️ Time taken: 0.00 seconds


'lovely123'

In [6]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.
